# Chapter 2 — An Array of Sequences

**Sections with code snippets in this chapter:**

* [List Comprehensions and Generator Expressions](#List-Comprehensions-and-Generator-Expressions)
* [Tuples Are Not Just Immutable Lists](#Tuples-Are-Not-Just-Immutable-Lists)
* [Unpacking sequences and iterables](#Unpacking-sequences-and-iterables)
* [Pattern Matching with Sequences](#Pattern-Matching-with-Sequences)
* [Slicing](#Slicing)
* [Using + and * with Sequences](#Using-+-and-*-with-Sequences)
* [Augmented Assignment with Sequences](#Augmented-Assignment-with-Sequences)
* [list.sort and the sorted Built-In Function](#list.sort-and-the-sorted-Built-In-Function)
* [When a List Is Not the Answer](#When-a-List-Is-Not-the-Answer)
* [Memory Views](#Memory-Views)
* [NumPy and SciPy](#NumPy-and-SciPy)
* [Deques and Other Queues](#Deques-and-Other-Queues)
* [Soapbox](#Soapbox)

## List Comprehensions and Generator Expressions

#### Example 2-1. Build a list of Unicode codepoints from a string

In [1]:
symbols = '$¢£¥€¤'
codes = []

for symbol in symbols:
    codes.append(ord(symbol))

codes

[36, 162, 163, 165, 8364, 164]

#### Example 2-2. Build a list of Unicode codepoints from a string, using a listcomp

In [2]:
symbols = '$¢£¥€¤'

codes = [ord(symbol) for symbol in symbols]

codes


[36, 162, 163, 165, 8364, 164]

#### Box: Listcomps No Longer Leak Their Variables

In [3]:
x = 'ABC'
codes = [ord(x) for x in x]
x

'ABC'

In [4]:
codes

[65, 66, 67]

In [5]:
codes = [last := ord(c) for c in x]
last

67

#### Example 2-3. The same list built by a listcomp and a map/filter composition

In [3]:
symbols = '$¢£¥€¤'
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
beyond_ascii

[162, 163, 165, 8364, 164]

In [7]:
beyond_ascii = list(filter(lambda c: c > 127, map(ord, symbols)))
beyond_ascii

[162, 163, 165, 8364, 164]

#### Example 2-4. Cartesian product using a list comprehension

In [6]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
tshirts = [(color, size) for color in colors for size in sizes]
tshirts

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

In [9]:
for color in colors:
    for size in sizes:
        print((color, size))

('black', 'S')
('black', 'M')
('black', 'L')
('white', 'S')
('white', 'M')
('white', 'L')


In [8]:
shirts = [(color, size) for size in sizes
          for color in colors]
shirts

[('black', 'S'),
 ('white', 'S'),
 ('black', 'M'),
 ('white', 'M'),
 ('black', 'L'),
 ('white', 'L')]

#### Example 2-5. Initializing a tuple and an array from a generator expression

In [8]:
symbols = '$¢£¥€¤'

In [13]:
%%timeit
# list comp: you get a list, greedy eval
# genexp: lazily evaluated, good for keeping memory usage down

tuple([ord(symbol) for symbol in symbols])


591 ns ± 61.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [12]:
%%timeit
tuple(ord(symbol) for symbol in symbols)

752 ns ± 20.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [15]:
genexp = (ord(symbol) for symbol in symbols)

next(genexp)
next(genexp)

162

In [12]:
import array

array.array('I', (ord(symbol) for symbol in symbols))

array('I', [36, 162, 163, 165, 8364, 164])

#### Example 2-6. Cartesian product in a generator expression

In [16]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']

for tshirt in ('%s %s' % (c, s) for c in colors for s in sizes):
    print(tshirt)

black S
black M
black L
white S
white M
white L


In [36]:
#list vs tuple
my_list = [e for e in "string"]
my_list[0] = "b"
my_list[1: 3]

# how to add to list
my_list.append("g")

my_list.extend([1,2,3])
my_list

my_list_plus = my_list + ['a', 'b', 'c']
my_list_plus

# tup = tuple(e for e in "string")
# tup[0] = "b"

# how to change a tuple? create a new one
sorted_names = ["Jamie", "John"]
name_to_class = {
    tuple(sorted_names): "python"
}
name_to_class

{('Jamie', 'John'): 'python'}

## Tuples Are Not Just Immutable Lists

#### Example 2-7. Tuples used as records

In [1]:

lax_coordinates = (33.9425, -118.408056)
city, year, pop, chg, area = ('Tokyo', 2003, 32_450, 0.66, 8014)


traveler_ids = [('USA', '31195855'), ('BRA', 'DEF123124'), ('BRA', 'CE342567'), ('ESP', 'XDA205856')]

for passport in sorted(traveler_ids):
    print('%s/%s' % passport)

BRA/CE342567
BRA/DEF123124
ESP/XDA205856
USA/31195855


In [15]:
for country, _ in traveler_ids:
    print(country)

USA
BRA
ESP


### Tuples as Immutable Lists

In [5]:
my_list = [1, 2]
a = (10, 'alpha', my_list)
b = (10, 'alpha', my_list)
a == b

True

In [6]:
b[-1].append(99)
a == b

True

In [8]:
b

(10, 'alpha', [1, 2, 99])

In [9]:
def fixed(o):
    try:
        hash(o)
    except TypeError:
        return False
    return True


tf = (10, 'alpha', (1, 2))  # Contains no mutable items
tm = (10, 'alpha', [1, 2])  # Contains a mutable item (list)
fixed(tf)

True

In [20]:
fixed(tm)

False

## Unpacking sequences and iterables

In [12]:
lax_coordinates = (33.9425, -118.408056)
latitude, longitude = lax_coordinates  # unpacking
latitude

33.9425

In [13]:
longitude

-118.408056

In [14]:
divmod(20, 8)

(2, 4)

In [16]:
t = (20, 8)
divmod(*t)  # equivalent to divmod(20, 8)

(2, 4)

In [17]:
quotient, remainder = divmod(*t)
quotient, remainder

(2, 4)

In [27]:
import os

_, filename = os.path.split('/home/luciano/.ssh/id_rsa.pub')
filename

'id_rsa.pub'

In [21]:
a = [1,2,3]
b = ['a', 'b', 'c']
for a_, b_ in zip(a, b):
    print(a_, b_)

a, b = zip(*zip(a,b))
b

1 a
2 b
3 c


('a', 'b', 'c')

### Using * to grab excess items

In [27]:
a, b, *rest = range(5)
a, b, rest

(0, 1, [2, 3, 4])

In [31]:
a, b, *rest = range(3)
a, b, rest

(0, 1, [2])

In [29]:
a, b, *rest = range(2)
a, b, rest

(0, 1, [])

In [36]:
a, *body, c, d = range(5)
a, body, c, d

(0, [1, 2], 3, 4)

In [33]:
*head, b, c, d = range(5)
head, b, c, d

([0, 1], 2, 3, 4)

### Unpacking with * in function calls and sequence literals

In [32]:
def fun(a, b, c, d, *rest):
    return a, b, c, d, rest


fun(*[1, 2], 3, *range(4, 7))

(1, 2, 3, 4, (5, 6))

In [33]:
*range(4), 4

(0, 1, 2, 3, 4)

In [34]:
[*range(4), 4]

[0, 1, 2, 3, 4]

In [37]:
{*range(4), 4, *(5, 6, 7)}

{0, 1, 2, 3, 4, 5, 6, 7}

### Nested unpacking
#### Example 2-8. Unpacking nested tuples to access the longitude

[02-array-seq/metro_lat_lon.py](02-array-seq/metro_lat_lon.py)

## Pattern Matching with Sequences
#### Example 2-9. Method from an imaginary Robot class

In [36]:
# def handle_command(self, message):
#     match message:
#         case ['BEEPER', frequency, times]:
#             self.beep(times, frequency)
#         case ['NECK', angle]:
#             self.rotate_neck(angle)
#         case ['LED', ident, intensity]:
#             self.leds[ident].set_brightness(ident, intensity)
#         case ['LED', ident, red, green, blue]:
#             self.leds[ident].set_color(ident, red, green, blue)
#         case _:
#             raise InvalidCommand(message)

#### Example 2-10. Destructuring nested tuples—requires Python ≥ 3.10.
[02-array-seq/match_lat_lon.py](02-array-seq/match_lat_lon.py)

In [37]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('São Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

def main():
    print(f'{"":15} | {"latitude":>9} | {"longitude":>9}')
    for record in metro_areas:
        match record:
            case [name, _, _, (lat, lon)] if lon <= 0:
                print(f'{name:15} | {lat:9.4f} | {lon:9.4f}')
main()

                |  latitude | longitude
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
São Paulo       |  -23.5478 |  -46.6358


### Pattern Matching Sequences in an Interpreter
#### Example 2-11. Matching patterns without match/case.
[02-array-seq/lispy/py3.9/lis.py](02-array-seq/lispy/py3.9/lis.py)

#### Example 2-12. Pattern matching with match/case—requires Python ≥ 3.10.
[02-array-seq/lispy/py3.10/lis.py](02-array-seq/lispy/py3.10/lis.py)

## Slicing

### Why Slices and Range Exclude the Last Item

In [39]:
l = [10, 20, 30, 40, 50, 60]
l[:2]  # split at 2

[10, 20]

In [42]:
l[2:]

[30, 40, 50, 60]

In [40]:
l[:3]  # split at 3

[10, 20, 30]

In [41]:
l[3:]

[40, 50, 60]

### Slice Objects

In [2]:
s = 'bicycle'
# s[start: stop: step]
s[::3]

'bye'

In [3]:
s[::-1]

'elcycib'

In [5]:
s[::-2]

'eccb'

#### Example 2-13. Line items from a flat-file invoice

In [6]:
invoice = """
0.....6.................................40........52...55........
1909 Pimoroni PiBrella                      $17.50    3    $52.50
1489 6mm Tactile Switch x20                  $4.95    2    $9.90
1510 Panavise Jr. - PV-201                  $28.00    1    $28.00
1601 PiTFT Mini Kit 320x240                 $34.95    1    $34.95
"""

SKU = slice(0, 6)
DESCRIPTION = slice(6, 40)
UNIT_PRICE = slice(40, 52)
QUANTITY = slice(52, 55)
ITEM_TOTAL = slice(55, None)

line_items = invoice.split('\n')[2:]

for item in line_items:
    print(item[UNIT_PRICE], item[DESCRIPTION])

    $17.50   imoroni PiBrella                  
     $4.95   mm Tactile Switch x20             
    $28.00   anavise Jr. - PV-201              
    $34.95   iTFT Mini Kit 320x240             
 


### Assigning to Slices

In [15]:
l = list(range(10))
l

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [16]:
l[2:5] = [20, 30]
l

[0, 1, 20, 30, 5, 6, 7, 8, 9]

In [17]:
del l[5:7]
l

[0, 1, 20, 30, 5, 8, 9]

In [19]:
l[3::2] = [11, 22]
l

[0, 1, 20, 11, 5, 22, 9]

By design, this example raises an exception::

In [22]:
try:
    l[2:5] = 100
except TypeError as e:
    print(repr(e))



TypeError('can only assign an iterable')


In [23]:
l[2:5] = [100]
l

[0, 1, 100]

## Using + and * with Sequences

In [24]:
l = [1, 2, 3]
l * 5

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]

In [25]:
5 * 'abcd'

'abcdabcdabcdabcdabcd'

In [27]:
l = [1, 2, 3] + [4, 5, 6]
# "1, 2, 34, 5, 6" <- js output. Woah
l

[1, 2, 3, 4, 5, 6]

In [30]:
s = "The cat" + " is asleep"
s

'The cat is asleep'

### Building Lists of Lists

#### Example 2-14. A list with three lists of length 3 can represent a tic-tac-toe board

In [31]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [34]:
from pprint import pprint
board = [['_'] * 3 for i in range(3)]
pprint(board)


[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]


In [35]:
board[1][2] = 'X'
board

[['_', '_', '_'], ['_', '_', 'X'], ['_', '_', '_']]

#### Example 2-15. A list with three references to the same list is useless

In [36]:
weird_board = [['_'] * 3] * 3
weird_board

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]

In [37]:
weird_board[1][2] = 'O'
weird_board

[['_', '_', 'O'], ['_', '_', 'O'], ['_', '_', 'O']]

#### Explanation

In [38]:
board = []
for i in range(3):
    row = ['_'] * 3
    board.append(row)
board

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]

In [39]:
board[2][0] = 'X'
board

[['_', '_', '_'], ['_', '_', '_'], ['X', '_', '_']]

## Augmented Assignment with Sequences

In [60]:
l = [1, 2, 3]
idl = id(l)

In [ ]:
# NBVAL_IGNORE_OUTPUT
idl

In [ ]:
l *= 2
l

In [ ]:
id(l) == idl  # same list

In [64]:
t = (1, 2, 3)
idt = id(t)

In [ ]:
# NBVAL_IGNORE_OUTPUT
idt

In [ ]:
t *= 2
id(t) == idt  # new tuple

### A += Assignment Puzzler
#### Example 2-16. A riddle

In [67]:
t = (1, 2, [30, 40])
try:
    t[2] += [50, 60]
except TypeError as e:
    print(repr(e))

TypeError("'tuple' object does not support item assignment")


#### Example 2-17. The unexpected result: item t2 is changed and an exception is raised

In [ ]:
t

#### Example 2-18. Bytecode for the expression s[a] += b

In [ ]:
import dis

dis.dis('s[a] += b')

## list.sort and the sorted Built-In Function

In [40]:
fruits = ['grape', 'raspberry', 'apple', 'banana']
sorted(fruits)

['apple', 'banana', 'grape', 'raspberry']

In [41]:
fruits

['grape', 'raspberry', 'apple', 'banana']

In [42]:
sorted(fruits, reverse=True)

['raspberry', 'grape', 'banana', 'apple']

In [47]:
sorted(fruits, key=lambda x: (len(x), str(x)), reverse=True)

['raspberry', 'banana', 'grape', 'apple']

In [46]:
sorted(fruits, key=len, reverse=True)

['raspberry', 'banana', 'grape', 'apple']

In [48]:
fruits

['grape', 'raspberry', 'apple', 'banana']

In [49]:
fruits.sort()
fruits

['apple', 'banana', 'grape', 'raspberry']

In [50]:
fruits

NameError: name 'sort' is not defined

## When a List Is Not the Answer

### Arrays

#### Example 2-19. Creating, saving, and loading a large array of floats

In [6]:
from array import array
from random import random, seed
seed(10)  # Use seed to make the output consistent

floats = array('d', (random() for i in range(10 ** 7)))
floats[10]

0.24999667668640035

In [7]:
with open('floats.bin', 'wb') as fp:
    floats.tofile(fp)

In [8]:
floats2 = array('d')

with open('floats.bin', 'rb') as fp:
    floats2.fromfile(fp, 10)

floats2[-1]

0.32777281162209315

In [10]:
floats[9]

0.32777281162209315

In [11]:
floats2 == floats

False

### Memory Views

#### Example 2-20. Handling 6 bytes memory of as 1×6, 2×3, and 3×2 views

In [ ]:
octets = array('B', range(6))
m1 = memoryview(octets)
m1.tolist()

In [ ]:
m2 = m1.cast('B', [2, 3])
m2.tolist()

In [ ]:
m3 = m1.cast('B', [3, 2])
m3.tolist()

In [ ]:
m2[1,1] = 22
m3[1,1] = 33
octets

#### Example 2-21. Changing the value of an 16-bit integer array item by poking one of its bytes

In [ ]:
numbers = array('h', [-2, -1, 0, 1, 2])
memv = memoryview(numbers)
len(memv)

In [ ]:
memv[0]

In [ ]:
memv_oct = memv.cast('B')
memv_oct.tolist()

In [ ]:
memv_oct[5] = 4
numbers

### NumPy

#### Example 2-22. Basic operations with rows and columns in a numpy.ndarray

In [13]:
import numpy as np

a = np.arange(12)
a

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [14]:
type(a)

numpy.ndarray

In [15]:
a.shape

(12,)

In [20]:
a.shape = 3, 4
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [25]:
b = a.reshape(12)
b

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [26]:
a[2]

array([ 8,  9, 10, 11])

In [27]:
a[2, 1]

9

In [28]:
a[:, 1]

array([1, 5, 9])

In [29]:
a.transpose()

array([[ 0,  4,  8],
       [ 1,  5,  9],
       [ 2,  6, 10],
       [ 3,  7, 11]])

#### Example 2-22. Loading, saving, and vectorized operations

In [30]:
with open('floats-1M-lines.txt', 'wt') as fp:
    for _ in range(1_000_000):
        fp.write(f'{random()}\n')

In [31]:
floats = np.loadtxt('floats-1M-lines.txt')

In [32]:
floats[-3:]

array([0.06078257, 0.61741189, 0.84349987])

In [33]:
floats *= .5
floats[-3:]

array([0.03039128, 0.30870594, 0.42174994])

In [34]:
from time import perf_counter as pc

t0 = pc()
floats /= 3
(pc() - t0) < 0.01

True

In [36]:
np.save('floats-1M', floats)
floats2 = np.load('floats-1M.npy', 'r+')
floats2 *= 6

In [37]:
floats2[-3:]

memmap([0.06078257, 0.61741189, 0.84349987])

### Deques and Other Queues

#### Example 2-23. Working with a deque

In [43]:
import collections

dq = collections.deque(range(10), maxlen=10)
dq

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], maxlen=10)

In [40]:
dq.append(11)
dq

deque([2, 3, 4, 5, 6, 7, 8, 9, 11, 11], maxlen=10)

In [44]:
dq.rotate(3)
dq

deque([7, 8, 9, 0, 1, 2, 3, 4, 5, 6], maxlen=10)

In [45]:
dq.rotate(-4)
dq

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 0], maxlen=10)

In [46]:
dq.appendleft(-1)
dq

deque([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9], maxlen=10)

In [47]:
dq.extend([11, 22, 33])
dq

deque([3, 4, 5, 6, 7, 8, 9, 11, 22, 33], maxlen=10)

In [48]:
dq.extendleft([10, 20, 30, 40])
dq

deque([40, 30, 20, 10, 3, 4, 5, 6, 7, 8], maxlen=10)

## Soapbox

### Mixed bag lists

In [110]:
l = [28, 14, '28', 5, '9', '1', 0, 6, '23', 19]

In [ ]:
try:
    sorted(l)
except TypeError as e:
    print(repr(e))

### Key is Brilliant

In [ ]:
l = [28, 14, '28', 5, '9', '1', 0, 6, '23', 19]

sorted(l, key=int)

In [ ]:
sorted(l, key=str)